In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
# ucitvanje podataka

data = pd.read_csv('./Google-Playstore.csv')
population = pd.read_csv('./Countries-Population.csv') # podaci iz 2018. godine

In [ ]:
# odbacivanje nepotrebnih podataka

data.drop('App Name', axis=1, inplace=True)
data.drop('App Id', axis=1, inplace=True)
data.drop('Scraped Time', axis=1, inplace=True)
data.drop('Developer Website', axis=1, inplace=True)
data.drop('Developer Email', axis=1, inplace=True)
data.drop('Privacy Policy', axis=1, inplace=True)
data.drop('Currency', axis=1, inplace=True)
data.drop('Installs', axis=1, inplace=True)
data.drop('Minimum Installs', axis=1, inplace=True)

In [ ]:
# Kodiranje

data['Free'].replace({True: 1, False: 0}, inplace=True)
data['Ad Supported'].replace({True: 1, False: 0}, inplace=True)
data['In App Purchases'].replace({True: 1, False: 0}, inplace=True)
data['Editors Choice'].replace({True: 1, False: 0}, inplace=True)

In [ ]:
# prikupljanje svih imena iz kolone "Category" i pravljenje novih kolona

list_of_categories = data['Category'].unique()
categories = pd.get_dummies(data['Category'])
categories.replace({True: 1, False: 0}, inplace=True)
data = pd.concat([data, categories], axis=1)
data.drop('Category', axis=1, inplace=True)

_ = pd.get_dummies(data['Content Rating'])
_.replace({True: 1, False: 0}, inplace=True)
data = pd.concat([data, _], axis=1)
data.drop('Content Rating', axis=1, inplace=True)

In [ ]:
# generisanje jedinstvenog ID za svakog developera

_ = pd.DataFrame()
_['Developer'] = data['Developer Id'].unique()
_['Id'] = range(1, len(_) + 1)

developers_dict = dict(zip(_['Developer'], _['Id']))

data['Developer Id'] = data['Developer Id'].map(developers_dict)

In [ ]:
# konvertovanje velicine aplikacije

def convert_to_mb(size):
    size_str = str(size)

    if (size_str[-1] == 'k'):
        return float(size_str[:-1].replace(',', '.')) / 1024

    if (size_str[-1] == 'M'):
        return float(size_str[:-1].replace(',', '.'))

    if (size_str[-1] == 'G'):
        return float(size_str[:-1]) * 1024.0

    else:
        return None

data['Size'] = data['Size'].apply(lambda x: convert_to_mb(x))

In [ ]:
# prevodjenje formata datuma

data['Released'] = pd.to_datetime(data['Released'])
data['Released'] = data['Released'].dt.strftime('%d.%m.%Y') 
data['Last Updated'] = pd.to_datetime(data['Last Updated'])
data['Last Updated'] = data['Last Updated'].dt.strftime('%d.%m.%Y') 

In [ ]:
# konvertovanje android verzija u datume

android_versions_dict = pd.read_csv('./Android-Versions.csv')
android_versions_dict['Android version'] = android_versions_dict['Android version'].apply(lambda x: str(x))
android_versions_dict = dict(zip(android_versions_dict['Android version'], android_versions_dict['Release date']))

def iseci(x):
    return str(x)[:3]

data['Minimum Android'] = data['Minimum Android'].apply(lambda x: iseci(x)) 

# Posto nemamo podatke o subverzijama androida kada su izasle, mora da gledamo samo verzije.
android_versions_dict.update({'nan': 0})
android_versions_dict.update({'Var': 'Varies with device'})

data['Minimum Android'] = data['Minimum Android'].map(android_versions_dict)

In [ ]:
# dopunjavanje NaN vrednosti
# Pretpostavljamo da aplikacija nije imala rating-ove

data['Rating'].fillna(0.0, inplace=True)
data['Rating Count'].fillna(0.0, inplace=True)

In [ ]:
# Nadoknadjivnje velicine aplikacije

X = categories.copy(deep=True)

y = data['Size']
missing_indexes = y[y.isnull()].index.to_list()
y.dropna(inplace=True)

X.drop(missing_indexes, inplace=True)

X_missing = categories.copy(deep=True)
X_missing = X_missing.iloc[missing_indexes]

In [ ]:
# fit-ovanje modela

knn_regressor = KNeighborsRegressor()
knn_regressor.fit(X,y)

In [ ]:
# nadoknadjivanje missing vrednosti

##### IZ NEPOZNATOG RAZLOGA PREVISE VREMENA TRAJE. 9MIN SAM GA PUSTIO I NIJE URADIO #####

y_missing = knn_regressor.predict()

In [ ]:
# SIZE
# RESITI NADOKNJADJIVANJE VELICINA APLIKACIJA
# MOZDA ODRADITI AVERAGE PO KATEGORIJAMA

# MINIMUM ANDROID
# RESITI SE "NaN" i "Varies with device" PODATAKA

# RELEASED
# RESITI SE "NaN" PODATAKA

# NAPRAVITI MODEL

# ZAVRSITI GUI - U TOKU VIKENDA CE BITI GOTOV
